In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import NA, NaT

### Leer .csv pasanti

In [ ]:
df_pasanti_sygnals=pd.read_csv("pasanti_sygnals.csv")

In [ ]:
df_sygnals=df_pasanti_sygnals.copy()
df_sygnals.drop(columns="Unnamed: 0",inplace=True)

In [ ]:
df_pasanti_sygnals["message_link"]

### Ajustar los tipos de datos

In [ ]:
# determinar que tipo de datos hay en el data frame.
df_sygnals.info()


In [ ]:
# convertir los datos tipo str a un typo de dato mas adecuado
df_sygnals=df_sygnals.convert_dtypes()

In [ ]:
df_sygnals.columns

### ajustar los datos de fechas

In [ ]:
# pasar los datos str a datetime
df_sygnals['date']=pd.to_datetime(df_sygnals['date'])

In [ ]:
# extraer la primera fecha del conjunto de fechas.
def extrat(x):
  if type(x) == type(NA):
    return x
  if x == "False":
    return False
  if x == "never":
    return "never"
  else:
    return pd.to_datetime(x)


In [ ]:
df_sygnals['first_dates_entry']=df_sygnals['first_dates_entry'].agg(lambda x: extrat(x))
df_sygnals['first_dates_stoploss']=df_sygnals['first_dates_stoploss'].agg(lambda x: extrat(x))
df_sygnals['first_dates_profit']=df_sygnals['first_dates_profit'].agg(lambda x: extrat(x))

## Calcular el tiempo transcurrido entre fechas

In [ ]:
# crear un df con las columnas de tiempo
df_times=df_sygnals.loc[:,["date","first_dates_profit","first_dates_stoploss","first_dates_entry"]]
df_times

In [ ]:
# funcion que calcula la diferencia entre dos fechas
def delta_time(d,e):
  d_e=False
  if type(e)==type(d) and type(e)!= bool and e is not NA:
    d_e=e-d
    d_e=d_e.total_seconds()
  return d_e

In [ ]:
# nuevas columnas de diferencias entre fechas
df_times["d_e"]=df_times.agg(lambda x: delta_time(x.date,x.first_dates_entry),axis=1)
df_times["d_p"]=df_times.agg(lambda x: delta_time(x.date,x.first_dates_profit),axis=1)
df_times["e_p"]=df_times.agg(lambda x: delta_time(x.first_dates_entry,x.first_dates_profit),axis=1)
df_times["d_s"]=df_times.agg(lambda x: delta_time(x.date,x.first_dates_stoploss),axis=1)

In [ ]:
#pd.concat([df_times,df_sygnals[[""]]]

In [ ]:
# para ver los segundo en deltatime
str(timedelta(seconds=153229.7503816794))
str(timedelta(seconds=233928.42055084746))
str(timedelta(seconds=36378571.0))

In [ ]:
# sns.set(rc={'figure.figsize':(20,120)},font_scale=1.7)
# sns.violinplot(data=df_times[["d_e"]])

In [ ]:
# Calcular los min, max, mena, median y mostrar en formato timedelta

pd.concat([
    (df_times[df_times["d_e"]!=False]["d_e"].agg([min,max,np.mean,np.median])).apply(lambda x: str(timedelta(seconds=x))),
    df_times[df_times["d_p"]!=False]["d_p"].agg([min,max,np.mean,np.median]).apply(lambda x: str(timedelta(seconds=x))),
    df_times[df_times["e_p"]!=False]["e_p"].agg([min,max,np.mean,np.median]).apply(lambda x: str(timedelta(seconds=x))),
    df_times[df_times["d_s"]!=False]["d_s"].agg([min,max,np.mean,np.median]).apply(lambda x: str(timedelta(seconds=x))),
],axis=1)

## Cantidad de señales por canal

In [ ]:
df_sygnals["channel"].replace(['𝑪𝒐𝒊𝒏|𝑪𝒐𝒂𝒄𝒉|𝑺𝒊𝒈𝒏𝒂𝒍𝒔','Crypto Futures Spot Signals😘'],["Coin|Coach|Signals","Crypto Futures Spot Signals"],inplace=True)

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
sns.set(rc={'figure.figsize':(20,10)},font_scale=1.7)
sns.histplot(df_sygnals,x="channel",hue="channel" , palette="Set2")


## eficiencia de señales por canal

In [ ]:
channels=df_sygnals["channel"].unique()
channels

In [ ]:
efficiency_by_channel=df_sygnals[["channel","efficiency","symbol"]].groupby("channel")

In [ ]:
sns.histplot(df_sygnals, x='efficiency', hue = 'channel', multiple="dodge")
plt.title("Distribucion de eficiencia por canal")


In [ ]:
sns.displot(df_sygnals, x='efficiency', hue = 'channel', multiple="dodge")
plt.title("Distribucion de eficiencia por canal")

## Vistazo de cantidad de operaciones con ganancia y sin ganancia

In [ ]:
is_profit:pd.Series=df_sygnals["efficiency"].dropna().agg(lambda x: "profit" if x > 0 else "no_profit")
is_profit.name="is_profit"

In [ ]:
df_isprofit=pd.concat([df_sygnals[["efficiency","channel","symbol","_id"]],is_profit],axis=1)

In [ ]:
sns.histplot(df_isprofit, x='is_profit', hue = 'channel', multiple="dodge",palette="Set2")
plt.title("Distribucion de profit por canal")

## Cantidad de operaciones por criptomoneda

### Cantidad de symbolos y pares

In [ ]:
df_sygnals["symbol"].unique()

In [ ]:
df_sygnals.columns

In [ ]:
df_sygnals['primary_currencies'].unique()


In [ ]:
df_sygnals['segundary_currencies'].unique()


In [ ]:
'Crypto Futures Spot Signals', 'Coin|Coach|Signals', 'Bitcoin Bullets VIP Free', 'Federal Russian Insiders VIP Free'

### 'Crypto Futures Spot Signals'

In [ ]:
#construir df para graficar

canal='Crypto Futures Spot Signals'

df_prf=df_isprofit.groupby(["channel"]).get_group(canal)
df_prf["quantity"]=df_prf.agg(lambda x: (dict(df_prf.value_counts("symbol"))[x["symbol"]]/len(df_prf["symbol"].unique()))*100,axis=1)
df_prf.sort_values("quantity",ascending=False)



sns.set(rc={'figure.figsize':(10,12)},font_scale=1)
sns.histplot(df_prf.sort_values("quantity",ascending=False).head(270), y="symbol", hue= 'is_profit', multiple="dodge",palette="Set1")

plt.title(canal)

print(f"""cantidad de symbolos {len(df_prf["symbol"].unique())}""")
print(f"""cantidad por symbolo {df_prf.value_counts("symbol").head(7)}""")


### 'Coin|Coach|Signals'

In [ ]:
#construir df para graficar

canal='Coin|Coach|Signals'

df_prf=df_isprofit.groupby(["channel"]).get_group(canal)
df_prf["quantity"]=df_prf.agg(lambda x: (dict(df_prf.value_counts("symbol"))[x["symbol"]]/len(df_prf["symbol"].unique()))*100,axis=1)
df_prf.sort_values("quantity",ascending=False)



sns.set(rc={'figure.figsize':(10,10)},font_scale=1)
sns.histplot(df_prf.sort_values("quantity",ascending=False).head(250), y="symbol", hue= 'is_profit', multiple="dodge",palette="Set1")

plt.title(canal)

print(f"""cantidad de symbolos {len(df_prf["symbol"].unique())}""")
print(f"""cantidad por symbolo {df_prf.value_counts("symbol").head(7)}""")

### 'Bitcoin Bullets VIP Free'

In [ ]:
#construir df para graficar

canal='Bitcoin Bullets VIP Free'

df_prf=df_isprofit.groupby(["channel"]).get_group(canal)
df_prf["quantity"]=df_prf.agg(lambda x: (dict(df_prf.value_counts("symbol"))[x["symbol"]]/len(df_prf["symbol"].unique()))*100,axis=1)
df_prf.sort_values("quantity",ascending=False)



sns.set(rc={'figure.figsize':(10,15)},font_scale=1)
sns.histplot(df_prf.sort_values("quantity",ascending=False), y="symbol", hue= 'is_profit', multiple="dodge",palette="Set1")

plt.title(canal)

print(f"""cantidad de symbolos {len(df_prf["symbol"].unique())}""")
print(f"""cantidad por symbolo {df_prf.value_counts("symbol").head(7)}""")

### 'Federal Russian Insiders VIP Free'

In [ ]:
#construir df para graficar

canal='Federal Russian Insiders VIP Free'

df_prf=df_isprofit.groupby(["channel"]).get_group(canal)
df_prf["quantity"]=df_prf.agg(lambda x: (dict(df_prf.value_counts("symbol"))[x["symbol"]]/len(df_prf["symbol"].unique()))*100,axis=1)
df_prf.sort_values("quantity",ascending=False)



sns.set(rc={'figure.figsize':(10,15)},font_scale=1)
sns.histplot(df_prf.sort_values("quantity",ascending=False), y="symbol", hue= 'is_profit', multiple="dodge",palette="Set1")

plt.title(canal)

print(f"""cantidad de symbolos {len(df_prf["symbol"].unique())}""")
print(f"""cantidad por symbolo {df_prf.value_counts("symbol").head(7)}""")

## variaciones de tiempos segun canal

In [ ]:
df_sygnals.columns,df_sygnals.shape
timedeltaP=(df_sygnals['first_dates_stoploss'].iloc[6]-df_sygnals['date'].iloc[6])

In [ ]:
df_isprofit.columns,df_isprofit["channel"].unique()

In [ ]:
df_times.columns,df_times.shape

In [ ]:
def delta_time_ca(x):
    return str(timedelta(seconds=x))

In [ ]:
df_sygnals_2=pd.concat([df_isprofit,df_times],axis=1)

In [ ]:
mask=(df_sygnals_2["is_profit"]=="profit") & (df_sygnals_2["channel"]=="Crypto Futures Spot Signals") & (df_times["d_e"]!=False)& (df_times["d_p"]!=False) & (df_times["e_p"]!=False) & (df_times["d_s"]!=False)
df_sygnals_2[mask][['d_e', 'd_p', 'e_p', 'd_s']].apply([max,min,np.mean, np.median])#.apply(lambda x : list(map(delta_time_ca,x)) )

In [ ]:
mask=(df_sygnals_2["is_profit"]=="profit") & (df_sygnals_2["channel"]=="Crypto Futures Spot Signals") & (df_times["d_e"]!=False)& (df_times["d_p"]!=False) & (df_times["e_p"]!=False) & (df_times["d_s"]!=False)
df_sygnals_2[mask][['d_e', 'd_p', 'e_p', 'd_s']].apply([max,min,np.mean, np.median]).apply(lambda x : list(map(delta_time_ca,x)) )

In [ ]:
str(timedelta(seconds=5.048409e+07))

In [ ]:
df_sygnals_2[mask][['d_e', 'd_p', 'e_p', 'd_s']].transform(lambda x: round(x/3600))

In [ ]:
df_sygnals_2[mask][['d_s']].max()

In [ ]:
round(50484089.0/3600)

In [ ]:
pd.concat([
    (df_times[df_times["d_e"]!=False]["d_e"].agg([min,max,np.mean,np.median])).apply(lambda x: str(timedelta(seconds=x))),
    df_times[df_times["d_p"]!=False]["d_p"].agg([min,max,np.mean,np.median]).apply(lambda x: str(timedelta(seconds=x))),
    df_times[df_times["e_p"]!=False]["e_p"].agg([min,max,np.mean,np.median]).apply(lambda x: str(timedelta(seconds=x))),
    df_times[df_times["d_s"]!=False]["d_s"].agg([min,max,np.mean,np.median]).apply(lambda x: str(timedelta(seconds=x))),
],axis=1)